In [4]:
import numpy as np
import pandas as pd
from scipy.integrate import solve_ivp
from matplotlib import pyplot as plt
from plasmistry.electron import EEDF
from plasmistry.electron import get_maxwell_eedf
from plasmistry.molecule import get_ideal_gas_density

In [5]:
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (12,9)
plt.rcParams["font.size"] = 14

In [6]:
np.set_printoptions(precision=1, linewidth=130, edgeitems=5)
pd.options.display.float_format = '{:,.1e}'.format

### Read inelas reactions

inelas_df_forward = pd.read_pickle('rctn_dataframe_forward.pkl')
inelas_df_backward = pd.read_pickle('rctn_dataframe_backward.pkl')

inelas_df_forward.loc[0, 'cross_section']

In [7]:
# inelas_df = pd.concat([inelas_df_forward, inelas_df_backward], ignore_index=True)
# inelas_df = pd.read_pickle("rctn_dataframe_H2_dis_via_b.pkl")
# inelas_df = pd.read_pickle("CO_ele_vib_forward_df.pkl")
inelas_df = pd.read_pickle("CO_ele_vib_backward_df.pkl")

FileNotFoundError: [Errno 2] No such file or directory: 'CO_ele_vib_backward_df.pkl'

In [0]:
inelas_df

In [0]:
inelas_df.loc[0, 'cross_section']

### Choose the reactions considered.

In [0]:
#inelas_df = inelas_df.iloc[[0,1,2,3,4,5,6,7,8]].reset_index(drop=True)
inelas_df

In [0]:
inelas_df.loc[0, 'cross_section']

In [0]:
eedf = EEDF(max_energy_eV=50, grid_number=500)

In [0]:
eedf.initialize(rctn_with_crostn_df=inelas_df, total_species=['E', 'CO'] + [f'CO(v{v})' for v in range(1, 11)])

In [0]:
eedf.rate_const_matrix_e_inelas_electron.toarray()

In [0]:
eedf.rate_const_matrix_e_inelas_electron.shape

In [0]:
eedf.rate_const_matrix_e_inelas_molecule.shape

In [0]:
plt.matshow(eedf.rate_const_matrix_e_inelas_molecule)

In [0]:
eedf.n_bg_molecule_elas

In [0]:
eedf.bg_molecule_elas

In [0]:
eedf.n_bg_molecule_inelas

In [0]:
eedf.bg_molecule_inelas

In [0]:
eedf.n_inelas_rctn

In [0]:
eedf.bg_molecule_inelas_in_order

In [0]:
plt.spy(eedf.rate_const_matrix_e_inelas_electron, markersize=2)

In [0]:
eedf.rate_const_matrix_e_inelas_molecule

In [0]:
N = get_ideal_gas_density(p_Pa=1e5, Tgas_K=1000)
N

### Set parameters

In [0]:
eedf.set_parameters(E=2e5, Tgas=1000, N=N)

In [0]:
total_species_density = np.ones(12) * N
# total_species_density = np.array([0,1,0,0,0])*N
def dndt_all(t, y):
    eedf.set_density_per_J(y)
    eedf.set_flux(total_species_density=total_species_density)
    return eedf.get_deriv_total(total_species_density=total_species_density) 

In [0]:
y0 = get_maxwell_eedf(eedf.energy_point, Te_eV=0.7)*1e20
time_span = [0, 1e3]
sol = solve_ivp(dndt_all, time_span, y0, method="BDF", rtol=5e-2)

In [0]:
sol.t

In [0]:
len(sol.t)

In [0]:
plt.semilogy(eedf.energy_point_eV, (sol.y.transpose()/np.sqrt(eedf.energy_point)).transpose()[:,::50])

In [0]:
eedf.set_density_per_J(sol.y[:,0])
eedf.plot_normalized_eepf()

In [0]:
eedf.plot_normalized_eedf()

In [0]:
eedf.set_density_per_J(sol.y[:,-1])
eedf.plot_normalized_eepf(xlim=(0,12), ylim=(1e-12, 1e1))

In [0]:
print(eedf)

In [0]:
eedf._get_molecule_rate_const_e_inelas()

In [0]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.barh(range(eedf.n_inelas_rctn),
        width=eedf._get_molecule_rate_const_e_inelas(),
        height=0.6, log=True)
ax.set_yticks(range(eedf.n_inelas_rctn))
ax.set_yticklabels(eedf.inelas_reaction_dataframe['formula'])

In [0]:
eedf.plot_normalized_eedf()

In [0]:
Te_seq = []
ne_seq = []
for _array in sol.y.transpose():
    eedf.set_density_per_J(_array)
    Te_seq.append(eedf.electron_temperature_in_eV)
    ne_seq.append(eedf.electron_density)
plt.semilogx(sol.t, Te_seq, marker='.')
plt.title("Te vs. t")

In [0]:
plt.semilogx(sol.t, ne_seq, marker='.')
plt.title("ne vs. t")

In [0]:
sol.y

In [0]:
eedf.set_density_per_J(sol.y[:,-1])
eedf.set_flux(total_species_density=total_species_density)

In [0]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(eedf.energy_point_eV, eedf._get_deriv_ef(), 
        linewidth=0.4, marker='.', label='ef')
ax.plot(eedf.energy_point_eV, eedf._get_deriv_el(), 
        linewidth=0.4, marker='.', label='el')
ax.plot(eedf.energy_point_eV, eedf._get_deriv_ee(), 
        linewidth=0.4, marker='.', label='ee')
ax.plot(eedf.energy_point_eV, eedf._get_electron_rate_e_inelas(density=total_species_density), 
        linewidth=0.4, marker='.', label='e_inelas')
ax.plot(eedf.energy_point_eV, eedf.get_deriv_total(total_species_density=total_species_density), 
        linewidth=1.5, marker='.', label='sum')
# ax.plot(eedf.energy_nodes,eedf.J_flux_el, marker='.', label='el')
# ax.plot(eedf.energy_nodes,eedf.J_flux_ee, marker='.', label='ee')
# ax.plot(eedf.energy_nodes,eedf.J_flux_ef+eedf.J_flux_el+eedf.J_flux_ee, marker='.', label='ef+el+ee')
# ax.set_yscale('symlog')
# ax.set_xscale('log')
ax.set_xlim((0,4))
ax.legend()

In [0]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(eedf.energy_point_eV, eedf._get_deriv_ef()*eedf.energy_point, 
        linewidth=0.4, marker='.', label='ef')
ax.plot(eedf.energy_point_eV, eedf._get_deriv_el()*eedf.energy_point, 
        linewidth=0.4, marker='.', label='el')
ax.plot(eedf.energy_point_eV, eedf._get_deriv_ee()*eedf.energy_point, 
        linewidth=0.4, marker='.', label='ee')
ax.plot(eedf.energy_point_eV, eedf._get_electron_rate_e_inelas(density=total_species_density)*eedf.energy_point, 
        linewidth=0.4, marker='.', label='e_inelas')
ax.plot(eedf.energy_point_eV, eedf.get_deriv_total(total_species_density=total_species_density)*eedf.energy_point, 
        linewidth=1.5, marker='.', label='sum')
# ax.plot(eedf.energy_nodes,eedf.J_flux_el, marker='.', label='el')
# ax.plot(eedf.energy_nodes,eedf.J_flux_ee, marker='.', label='ee')
# ax.plot(eedf.energy_nodes,eedf.J_flux_ef+eedf.J_flux_el+eedf.J_flux_ee, marker='.', label='ef+el+ee')
# ax.set_yscale('symlog')
# ax.set_xscale('log')
ax.set_xlim((0,4))
ax.legend()

In [0]:
energy_gain_ef = (eedf._get_deriv_ef()*eedf.energy_point).sum()
energy_lose_el = (eedf._get_deriv_el()*eedf.energy_point).sum()
energy_gain_ee = (eedf._get_deriv_ee()*eedf.energy_point).sum()
energy_gain_e_inelas = (eedf._get_electron_rate_e_inelas(density=total_species_density)*eedf.energy_point).sum()
energy_gain_net = (eedf.get_deriv_total(total_species_density=total_species_density)*eedf.energy_point).sum()

In [0]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.barh(range(5),
        width=[energy_gain_ef, energy_lose_el, energy_gain_ee, energy_gain_e_inelas, energy_gain_net],
       height=0.6)
ax.set_yticks(range(5))
ax.set_yticklabels(['ef', 'el', 'ee', 'inelas', 'net'])
ax.set_title("energy lose")

In [0]:
eedf.inelas_rctn_info

In [0]:
eedf.inelas_rctn_info.so